In [2]:
import numpy as np
import h5py
import scipy.io


In [3]:
# Loads KRAKEN data and returns, training, hold out validation, and test
def load_dataset(idx, shuffle, Ntrain):
    filepath = "files_116/files_VLA_rand_"+ str(idx) +"_116.mat"
    
    mat = scipy.io.loadmat(filepath)
    signal_train=np.array([], dtype=np.float)
  
    # Training data
    args = (mat['p_cl_n'], mat['p_si_n'], mat['p_sa_n'],mat['p_gr_n'])
    for v in args: 
        tmp=np.vstack([np.real(v), np.imag(v)]) 
        signal_train=np.hstack([signal_train, tmp]) if signal_train.size else tmp

    mats=[1500,  1575, 1650, 1800]
    labels_train=np.array([mats[int(np.floor(i/1000))] for i in range(4000)], dtype=np.float)
    
    # Test data
    signal_test=np.array([],dtype=np.float)
    labels_test=np.array([], dtype=np.float)
    
    for j in range(10):
        i=j+1
        args=(mat["p_cl_n"+str(i)], mat['p_si_n'+str(i)], mat["p_sa_n"+str(i)], mat["p_gr_n"+str(i)])
        for v in args: 
            tmp=np.vstack([np.real(v), np.imag(v)])
            signal_test=np.hstack([signal_test, tmp]) if signal_test.size else tmp

        labels_test=np.real(np.append([labels_test], [labels_train]))

    # Test labels are perturbed, associate them with the correct material type
    Y=labels_test
    labels=labels_test
    
    X = signal_train.transpose()
    y = labels_train
    X_test = signal_test.transpose()
    y_test = Y
    
    X = signal_train.transpose()
    y = labels_train
    X_test = signal_test.transpose()
    y_test = labels_test
    
   
    ind=range(len(y))
    ind_train=ind[1:Ntrain]
    ind_val=ind[Ntrain:len(y)]
    ind_test = range(len(y_test))

  
    if shuffle:
        ind1 = np.random.permutation(len(y))
        ind_train=ind1[0:Ntrain]
        ind_val=ind1[Ntrain:len(y)]
        ind_test = np.random.permutation(len(y_test))
    
    X_train = np.array(X[ind_train,:], dtype = np.float)
    y_train = np.array(y[ind_train],)   
    X_val = np.array(X[ind_val,:], dtype = np.float)
    y_val = np.array(y[ind_val],) 
    X_test = np.array(X_test[ind_test,:], dtype = np.float)
    y_test = np.array(y_test[ind_test],)  
    return X_train, y_train, X_val, y_val, X_test, y_test



In [4]:
# Plot the training and test data
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm


X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(15,False, 3200)

f, ax = plt.subplots(2, 2, sharey=False, sharex=False)
ax[0][0].plot(X_train[:,4], color='orange')
ax[0][1].plot(y_train, color='yellow')
ax[1][0].plot(X_test[0:4000,20], color='green')
ax[1][1].plot(y_test, color='blue')

In [7]:
from time import time
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Benchmark metrics for each classifier
# Adapted from Peter Prettenhofer, et. al,
# https://scikit-learn.org/0.19/auto_examples/text/document_classification_20newsgroups.html
def benchmark(clf):
    target_names=[]
    targets=np.unique(y_train);

    for i in range(len(targets)):
        target_names.append(np.str(targets[i]))
    print('_' * 80)
    print("Training: ")
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)
    
    results=clf.cv_results_
    candidates = np.flatnonzero(results['rank_test_score'] == 1)
    for candidate in candidates:
        print("Model with rank: {0}".format(1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})"
              .format(results['mean_test_score'][candidate],
                      results['std_test_score'][candidate]))
        print("Parameters: {0}".format(results['params'][candidate]))
        print("\n")
    print("Validation: ")
    pred=clf.predict(X_val)
    test_time = time() - t0
    #print(metrics.confusion_matrix(y_val, pred))
    #print(metrics.classification_report(y_val, pred, target_names=target_names))
    score = metrics.accuracy_score(y_val, pred)
    print("val score:   %0.3f" % score)
    
    print("Testing: ")
    t0 = time()
    pred = clf.predict(X_test)
    print("test time:  %0.3fs" % test_time)
    
    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)
    print("classification report:")
    #print(metrics.classification_report(y_test, pred, target_names=target_names))
    print("confusion matrix:")
    print(metrics.confusion_matrix(y_test, pred))

    clf_descr = str(str(clf.estimator))
    clf_rep = str(metrics.classification_report(y_test, pred, target_names=target_names))
    clf_cm = str(metrics.confusion_matrix(y_test, pred))
    return clf_descr, score, train_time, test_time, clf_rep, clf_cm
      

In [8]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform, expon
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis


# list of (estimator, param_dist), where param_dist is used  in RandomizedSearchCV
classifiers = [
#      (GaussianProcessClassifier(warm_start=True, n_jobs=-1, random_state=42),{
#         'multi_class': ["one_vs_rest", "one_vs_one"],
#         'n_restarts_optimizer':[0, 1]
#      }),
     (KNeighborsClassifier(), {
         'n_neighbors': sp_randint(8, 10),
         'weights': ['uniform', 'distance'],
         'algorithm':['ball_tree', 'kd_tree'],
     }),
    (NearestCentroid(), {
         'metric': ['euclidean', 'manhattan','minkowski','chebyshev'],
         'shrink_threshold': [None, .001, .0001, .1, .01]
     }),
    (SVC(kernel='linear'), {
        'C': np.logspace(-1, 3, 100),
        'tol': np.logspace(-8, -2, 100)
    }),
    (SVC(random_state=42, gamma='scale'), {
        'C': [.1, 1, 10, 100, 1000],
        'kernel': ['rbf', 'poly', 'linear', 'sigmoid'], 
        'class_weight':['balanced', None],
        'tol': np.logspace(-8, -2, 100)
    }),
    (MLPClassifier(max_iter=100000), {
        'hidden_layer_sizes': [(200), (50), (100,)],
        'alpha': expon(scale=.1),
        'learning_rate': ['constant','adaptive'],
    }),
    (DecisionTreeClassifier(),{
        'criterion': ['gini', 'entropy'],
        'splitter': ['best','random'],
        'max_features':[None, 'auto', 'sqrt', 'log2'],
    }),
    (RandomForestClassifier(max_depth=5, n_jobs=-1), {
        "max_depth": sp_randint(2, 20),
        "n_estimators": sp_randint(2, 50),
        "max_features": ['auto', 'log2'],
        'class_weight': ['balanced', 'balanced_subsample'],
        'criterion': ['gini', 'entropy']
    }),
     (GaussianNB(), {
         'var_smoothing': np.logspace(-12, -8, 100)
     }), 
     (LinearDiscriminantAnalysis(n_components=2), {
         'solver': ['svd', 'lsqr'],
         'tol': np.logspace(-10, -2, 100)
     }),
     (LogisticRegression(solver='newton-cg', random_state=0, max_iter=100000), {
        'C': [.1, 1, 10, 100, 1000],
         'multi_class': ["auto", "ovr","multinomial"],
         'solver': ['sag', 'saga', 'newton-cg'],
         'tol': np.logspace(-6, -4, 100)
     }),
]
names = [e.__class__.__name__ for e, g in classifiers]

In [9]:
# tests the classifiers without hyperparameter search
X_train, y_train,X_val, y_val, X_test, y_test= load_dataset(15, True, 3999)
X_val=X_train
y_val=y_train
target_names=[]
targets=np.unique(y_train);
for i in range(len(targets)):
        target_names.append(np.str(targets[i]))
        

j=1
(clf, param_grid)=classifiers[j]
name=names[j]
print(name)
clf.fit(X_train, y_train)
pred=clf.predict(X_val)
print(metrics.confusion_matrix(y_val, pred))
#print(metrics.classification_report(y_val, pred, target_names=target_names))
score = metrics.accuracy_score(y_val, pred)
print("val score:   %0.3f" % score)
pred=clf.predict(X_test)
print(metrics.confusion_matrix(y_test, pred))
#print(metrics.classification_report(y_test, pred, target_names=target_names))
score = metrics.accuracy_score(y_test, pred)
print("test score:   %0.3f" % score)

NearestCentroid
[[ 995    5    0    0]
 [   6  994    0    0]
 [   0    0 1000    0]
 [   0    0    0  999]]
val score:   0.997
[[ 2562  7406    31     1]
 [   11  9935    54     0]
 [    0    71  9929     0]
 [    0     0     0 10000]]
test score:   0.811


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split


data_name=['KRAKEN 2 layer']
dset_name=data_name

noiselevel = np.array([15, 17 ,19, 21, 26, 30, 32 ,33, 60])
grid_searches = [dict() for x in range(len(noiselevel))]
results = [[] for x in range(len(noiselevel))]
ds_cnt=-1
for nl in noiselevel:
    ds_cnt=ds_cnt+1
    X_train, y_train,X_val, y_val, X_test, y_test= load_dataset(nl, True, 3200)
    
    print('\n Noise level:', str(nl),'\n')
   
    # iterate over classifiers
    for est_idx, (name, (estimator, param_grid)) in \
        enumerate(zip(names, classifiers)):

        # Perform randomized grid search over possible hyperparameters
        clf = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, n_iter=20, cv=5,
                                 verbose=0, n_jobs=-1)
        
        with ignore_warnings(category=ConvergenceWarning):
            results[ds_cnt].append(benchmark(clf))
        grid_searches[ds_cnt][name] = clf

        # Make predictions for the four test cases
        print(name)
        
        #print('Test Score', str(clf.score(X_test, y_test)))
      
        
    


 Noise level: 15 

________________________________________________________________________________
Training: 
train time: 1.905s
Model with rank: 1
Mean validation score: 0.996 (std: 0.002)
Parameters: {'algorithm': 'kd_tree', 'n_neighbors': 9, 'weights': 'uniform'}


Model with rank: 1
Mean validation score: 0.996 (std: 0.002)
Parameters: {'algorithm': 'kd_tree', 'n_neighbors': 9, 'weights': 'uniform'}


Model with rank: 1
Mean validation score: 0.996 (std: 0.002)
Parameters: {'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}


Model with rank: 1
Mean validation score: 0.996 (std: 0.002)
Parameters: {'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}


Model with rank: 1
Mean validation score: 0.996 (std: 0.002)
Parameters: {'algorithm': 'kd_tree', 'n_neighbors': 9, 'weights': 'uniform'}


Model with rank: 1
Mean validation score: 0.996 (std: 0.002)
Parameters: {'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}


Model with rank: 1
M

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


train time: 217.971s
Model with rank: 1
Mean validation score: 0.995 (std: 0.003)
Parameters: {'alpha': 0.015046192029279143, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}


Validation: 
val score:   0.999
Testing: 
test time:  217.976s
accuracy:   0.798
classification report:
confusion matrix:
[[ 2331  7650    18     1]
 [   43  9616   341     0]
 [    0    13  9987     0]
 [    0     0     0 10000]]
MLPClassifier
________________________________________________________________________________
Training: 


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


train time: 0.301s
Model with rank: 1
Mean validation score: 0.951 (std: 0.008)
Parameters: {'splitter': 'best', 'max_features': None, 'criterion': 'entropy'}


Validation: 
val score:   0.961
Testing: 
test time:  0.302s
accuracy:   0.787
classification report:
confusion matrix:
[[3329 5533 1128   10]
 [ 346 8919  723   12]
 [  50  624 9286   40]
 [   3   51   12 9934]]
DecisionTreeClassifier
________________________________________________________________________________
Training: 
train time: 11.416s
Model with rank: 1
Mean validation score: 0.991 (std: 0.004)
Parameters: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 19, 'max_features': 'log2', 'n_estimators': 32}


Model with rank: 1
Mean validation score: 0.991 (std: 0.003)
Parameters: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 19, 'max_features': 'log2', 'n_estimators': 45}


Validation: 
val score:   0.995
Testing: 
test time:  11.522s
accuracy:   0.821
classification report:
confusion mat

...

train time: 2.698s
Model with rank: 1
Mean validation score: 0.733 (std: 0.010)
Parameters: {'tol': 2.477076355991714e-07, 'C': 911.1627561154896}


Validation: 
val score:   0.698
Testing: 
test time:  2.761s
accuracy:   0.633
classification report:
confusion matrix:
[[3749 3843 1678  730]
 [2143 5489 1918  450]
 [ 651 1758 7115  476]
 [ 255  435  348 8962]]
SVC
________________________________________________________________________________
Training: 
train time: 2.814s
Model with rank: 1
Mean validation score: 0.738 (std: 0.014)
Parameters: {'tol': 0.0012328467394420659, 'kernel': 'rbf', 'class_weight': 'balanced', 'C': 0.1}


Model with rank: 1
Mean validation score: 0.738 (std: 0.014)
Parameters: {'tol': 2.477076355991714e-07, 'kernel': 'rbf', 'class_weight': 'balanced', 'C': 0.1}


Validation: 
val score:   0.699
Testing: 
test time:  2.907s
accuracy:   0.633
classification report:
confusion matrix:
[[3685 3839 1700  776]
 [2109 5502 1918  471]
 [ 628 1770 7131  471]
 [ 238  413 

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


train time: 214.072s
Model with rank: 1
Mean validation score: 0.733 (std: 0.011)
Parameters: {'alpha': 0.004912250531199668, 'hidden_layer_sizes': 200, 'learning_rate': 'constant'}


Validation: 
val score:   0.694
Testing: 
test time:  214.074s
accuracy:   0.631
classification report:
confusion matrix:
[[4133 3365 1779  723]
 [2506 4919 2132  443]
 [ 746 1540 7284  430]
 [ 312  402  398 8888]]
MLPClassifier
________________________________________________________________________________
Training: 


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


train time: 0.366s
Model with rank: 1
Mean validation score: 0.487 (std: 0.016)
Parameters: {'splitter': 'best', 'max_features': None, 'criterion': 'gini'}


Validation: 
val score:   0.466
Testing: 
test time:  0.367s
accuracy:   0.441
classification report:
confusion matrix:
[[3117 3151 2312 1420]
 [2482 3551 2667 1300]
 [1464 2612 4495 1429]
 [ 881 1479 1167 6473]]
DecisionTreeClassifier
________________________________________________________________________________
Training: 
train time: 12.523s
Model with rank: 1
Mean validation score: 0.655 (std: 0.011)
Parameters: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 14, 'max_features': 'auto', 'n_estimators': 41}


Validation: 
val score:   0.636
Testing: 
test time:  12.629s
accuracy:   0.566
classification report:
confusion matrix:
[[3751 3296 1819 1134]
 [2589 4033 2495  883]
 [ 906 1772 6423  899]
 [ 431  616  538 8415]]
RandomForestClassifier
_______________________________________________________________________

In [13]:
import datetime
# Displaying results to paste into latex
for nc in range(len(names)):
    print("\\begin{filecontents}{"+names[nc]+"-kraken.dat}\n noise 	snr accuracy") 
    for nl in range(len(noiselevel)):
        clf_descr, score, train_time, test_time, clf_rep, clf_cm=results[nl][nc]
    
        print(str(noiselevel[nl])+" "+ str(18-nl)+" "+ str(score*100))
    print("\end{filecontents}")
    

\begin{filecontents}{KNeighborsClassifier-kraken.dat}
 noise 	snr accuracy
15 18 81.75
17 17 82.0775
19 16 81.8925
21 15 81.5025
26 14 78.6125
30 13 75.84
32 12 78.0025
33 11 73.91
60 10 55.6775
\end{filecontents}
\begin{filecontents}{NearestCentroid-kraken.dat}
 noise 	snr accuracy
15 18 81.04249999999999
17 17 81.13749999999999
19 16 80.61
21 15 80.8275
26 14 79.72749999999999
30 13 78.365
32 12 78.11749999999999
33 11 76.99249999999999
60 10 63.417500000000004
\end{filecontents}
\begin{filecontents}{SVC-kraken.dat}
 noise 	snr accuracy
15 18 84.93
17 17 77.33
19 16 80.705
21 15 81.525
26 14 79.59
30 13 77.07000000000001
32 12 78.435
33 11 77.0725
60 10 63.287499999999994
\end{filecontents}
\begin{filecontents}{SVC-kraken.dat}
 noise 	snr accuracy
15 18 80.975
17 17 80.925
19 16 80.25750000000001
21 15 81.23
26 14 79.5775
30 13 78.2925
32 12 78.03750000000001
33 11 76.5275
60 10 63.2575
\end{filecontents}
\begin{filecontents}{MLPClassifier-kraken.dat}
 noise 	snr accuracy
15 18 79.83

In [ ]:
# Displaying results in a pandas dataframe
#Code adapted from https://www.kaggle.com/grfiv4/displaying-the-results-of-a-grid-search
import pandas as pd
def score_summary(grid_searches, sort_by='mean_test_score'):
        frames = []
        for name, grid_search in grid_searches.items():
            frame = pd.DataFrame(grid_search.cv_results_)
            frame = frame.filter(regex='^(?!.*param_).*$')
            frame['estimator'] = len(frame)*[name]
            frames.append(frame)
        df = pd.concat(frames)
        
        df = df.sort_values([sort_by], ascending=False)
        df = df.reset_index()
        df = df.drop(['rank_test_score', 'index'], 1)
        
        columns = df.columns.tolist()
        columns.remove('estimator')
        columns = ['estimator']+columns
        df = df[columns]
        return df

In [ ]:
df=score_summary(grid_searches[0])
print(noiselevel[0])
df